# Ensemble XAI on BINN
This tutorial demonstrates how to train a BINN on a simulation dataset and
compute layer explanations using six different methods: **DeepLift**, **IntegratedGradients**, **GradientShap**, **Input×Gradient**, **SmoothGrad**, and **DeepLiftShap**.

In [ ]:
from pathlib import Path
import analysis.importance_calculation as ic
from analysis.train_original import load_reactome_once, train_dataset
from openbinn.explainer import Explainer

reactome = load_reactome_once()
scenario = Path('./data/b1_g1/1')  # beta=1.0, gamma=1.0

# Train BINN model on this scenario and save weights
train_dataset(scenario, reactome, best_params=None)


In [ ]:
# Compute explanations for each method
methods = ['deeplift', 'ig', 'gradshap', 'itg', 'sg', 'shap']
for method in methods:
    print(f'Running {method} ...')
    ic.METHOD = method
    ic.explain_dataset(scenario, reactome)
    # Alternatively instantiate Explainer directly:
    # model = ...  # load trained model
    # explainer = Explainer(method, model, {'baseline': 0})
    # attributions = explainer.get_layer_explanations(input_tensor)
